In [1]:
from stattlepy import Stattleship
import pandas as pd

In [ ]:
# Early Testing of API
New_query = Stattleship()
Token = New_query.set_token('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Querying by Team by Season
Output = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id='nfl-2017-2018')
Output

# Creating Function to help query data game-by-game
def func(s):
    s = 'Titans vs Cardinals December 10, 2017 at  4:05pm'
    s.split(' ')
    season_dict = {{'September', '2015'}: 'nfl-2015-2016', {'October', '2015'}: 'nfl-2015-2016', {'November', '2015'}: 'nfl-2015-2016', {'December', '2015'}: 'nfl-2015-2016', {'January', '2016'}: 'nfl-2015-2016', {'February', '2016'}: 'nfl-2015-2016', {'September', '2016'}: 'nfl-2016-2017', {'October', '2016'}: 'nfl-2016-2017', {'November', '2016'}: 'nfl-2016-2017', {'December', '2016'}: 'nfl-2016-2017', {'January', '2017'}: 'nfl-2016-2017', {'February', '2017'}: 'nfl-2016-2017', {'September', '2017'}: 'nfl-2017-2018', {'October', '2017'}: 'nfl-2017-2018', {'November', '2017'}: 'nfl-2017-2018', {'December', '2017'}: 'nfl-2017-2018', {'January', '2018'}: 'nfl-2017-2018', {'February', '2018'}: 'nfl-2017-2018'}
    l = s.split(' ')
    e1 = season_dict[l [0]]
    team_dict = {'Cardinals': 'ari', 'Falcons': 'atl', 'Ravens': 'bal', 'Bills': 'buf', 'Panthers': 'car', 'Bears': 'chi', 'Bengals': 'cin', 'Browns': 'cle', 'Cowboys': 'dal', 'Broncos': 'den', 'Lions': 'det', 'Packers': 'gb', 'Texans': 'hou', 'Colts': 'ind', 'Jaguars': 'jac', 'Chiefs': 'kc', 'Chargers': 'lac', 'Rams': 'stl', 'Dolphins': 'mia', 'Vikings': 'min', 'Patriots': 'ne', 'Saints': 'no', 'Giants': 'nyg', 'Jets': 'nyj', 'Raiders': 'oak', 'Eagles': 'phi', 'Steelers': 'pit', '49ers': 'sf', 'Seahawks': 'sea', 'Buccaneers': 'tb', 'Titans':'ten', 'Redskins': 'was'}
    team_dict[l [0]]
    e2 = team_dict[l [0]]
    e3 = team_dict[l [2]]
    month_dict = {'September': '09', 'October': '10', 'November': '11', 'December': '12', 'January': '01', 'February': '02'}
    e4 = month_dict[l [0]]
    time_dict = {'9:30am': '0930', '12:30pm': '1230', '1:00pm': '1300', '4:05pm': '1605', '4:25pm': '1625', '4:30pm': '1630', '7:10pm': '1910', '8:15pm': '2015', '8:20pm': '2020', '10:20pm': '2220'}
    e5 = time_dict[l [0]]
    return e1 + '-' + e2 + '-' + e3 + '-' + str(s.get(5)) + '-' + e4 + '-' + str(s.get(4)) + '-' + e5
goal " nfl-2017-2018-ten-ari-2017-12-10-1605"

In [ ]:
# Querying by Team by Game (only shows one team's stats)
Output_two = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id='nfl-2017-2018',team_id='nyg')
Output_two

In [4]:
# Querying by Team by Game (shows both teams' stats)
Output_three = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id='nfl-2017-2018', game_id='nfl-2017-2018-ari-det-2017-09-10-1300')
Output_three

Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2017-2018&game_id=nfl-2017-2018-ari-det-2017-09-10-1300&page=1
Stattleship API request complete


[{u'away_teams': [{u'abbreviation': u'ARI',
    u'color': u'97233F',
    u'colors': [u'97233F', u'000000', u'FFB612', u'A5ACAF'],
    u'created_at': u'2015-11-28T22:33:56-05:00',
    u'division_id': u'70c14524-5ddd-432b-8728-9c5f9af9b565',
    u'hashtag': u'AZCardinals',
    u'hashtags': [u'AZCardinals', u'BirdGang'],
    u'id': u'77c57737-b688-4bbf-a127-7b7f0ef8bcdd',
    u'latitude': None,
    u'league_id': u'ebb52980-4b25-4384-bb32-26d354568f0c',
    u'location': u'Arizona',
    u'longitude': None,
    u'name': u'Arizona',
    u'nickname': u'Cardinals',
    u'slug': u'nfl-ari',
    u'updated_at': u'2016-08-16T03:55:52-04:00'}],
  u'games': [{u'at_neutral_site': None,
    u'attendance': None,
    u'away_team_id': u'77c57737-b688-4bbf-a127-7b7f0ef8bcdd',
    u'away_team_outcome': u'loss',
    u'away_team_score': 23,
    u'broadcast': u'FOX',
    u'clock': u':00',
    u'clock_secs': 0,
    u'created_at': u'2017-04-22T16:24:42-04:00',
    u'daytime': True,
    u'duration': None,
    u'e